# 🫁 TB Detection with AST - Complete with Visualizations

**Train, visualize, and explore your TB detector with Grad-CAM explanations!**

This notebook includes:
- ✅ Complete training with proven AST
- ✅ Comprehensive visualizations
- ✅ Interactive Grad-CAM exploration
- ✅ Ready for GitHub + Hugging Face deployment

## Part 1: Training (Run Steps 1-9 from previous notebook)

After training completes, continue with the visualization steps below...

## Step 10: Create Comprehensive Visualizations

In [ ]:
# Create visualization script
import textwrap

viz_script = textwrap.dedent('''
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Read metrics
df = pd.read_csv('checkpoints_tb_ast/metrics_ast.csv')

# Fix accuracy scale
if df['val_acc'].max() > 1:
    df['val_acc'] = df['val_acc'] / 100

print(f"📊 Loaded {len(df)} epochs")
print(f"   Best accuracy: {df['val_acc'].max()*100:.2f}%")
print(f"   Avg energy savings: {df['energy_savings'].mean():.2f}%")

Path('visualizations').mkdir(exist_ok=True)

# 4-Panel Results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('TB Detection with AST - Comprehensive Results', fontsize=18, fontweight='bold')

# Loss
axes[0,0].plot(df['epoch'], df['train_loss'], 'o-', linewidth=2, color='#e74c3c', markersize=4)
axes[0,0].set_xlabel('Epoch', fontweight='bold')
axes[0,0].set_ylabel('Training Loss', fontweight='bold')
axes[0,0].set_title('Training Loss', fontweight='bold')
axes[0,0].grid(alpha=0.3)

# Accuracy
axes[0,1].plot(df['epoch'], df['val_acc']*100, 'o-', linewidth=2, color='#2ecc71', markersize=4)
axes[0,1].axhline(df['val_acc'].max()*100, color='red', linestyle='--', linewidth=2,
                  label=f'Best: {df["val_acc"].max()*100:.2f}%')
axes[0,1].set_xlabel('Epoch', fontweight='bold')
axes[0,1].set_ylabel('Validation Accuracy (%)', fontweight='bold')
axes[0,1].set_title('Validation Accuracy', fontweight='bold')
axes[0,1].legend()
axes[0,1].grid(alpha=0.3)

# Activation
axes[1,0].plot(df['epoch'], df['activation_rate']*100, 'o-', linewidth=2, color='#3498db', markersize=4)
avg_act = df[df['epoch'] > 2]['activation_rate'].mean()*100
axes[1,0].axhline(avg_act, color='purple', linestyle='--', label=f'Avg: {avg_act:.1f}%')
axes[1,0].set_xlabel('Epoch', fontweight='bold')
axes[1,0].set_ylabel('Activation Rate (%)', fontweight='bold')
axes[1,0].set_title('Sample Activation Rate', fontweight='bold')
axes[1,0].legend()
axes[1,0].grid(alpha=0.3)

# Energy
axes[1,1].plot(df['epoch'], df['energy_savings'], 'o-', linewidth=2, color='#27ae60', markersize=4)
avg_savings = df[df['epoch'] > 2]['energy_savings'].mean()
axes[1,1].axhline(avg_savings, color='red', linestyle='--', label=f'Avg: {avg_savings:.1f}%')
axes[1,1].set_xlabel('Epoch', fontweight='bold')
axes[1,1].set_ylabel('Energy Savings (%)', fontweight='bold')
axes[1,1].set_title('Energy Savings', fontweight='bold')
axes[1,1].legend()
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('visualizations/tb_ast_results.png', dpi=300, bbox_inches='tight', facecolor='white')
print("✅ Saved: tb_ast_results.png")
plt.close()

# Social Media Headline
fig, ax = plt.subplots(figsize=(12, 8))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#16213e')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

best_acc = df['val_acc'].max() * 100

ax.text(5, 8.5, '🫁 TB Detection with AST', ha='center', fontsize=32, fontweight='bold', color='white')

box = dict(boxstyle='round,pad=0.8', facecolor='#0f3460', edgecolor='#00d4ff', linewidth=3)

ax.text(2.5, 6.5, f'{best_acc:.1f}%', ha='center', fontsize=48, fontweight='bold',
        color='#2ecc71', bbox=box)
ax.text(2.5, 5.5, 'Accuracy', ha='center', fontsize=16, color='white')

ax.text(7.5, 6.5, f'{avg_savings:.1f}%', ha='center', fontsize=48, fontweight='bold',
        color='#f39c12', bbox=box)
ax.text(7.5, 5.5, 'Energy Savings', ha='center', fontsize=16, color='white')

ax.text(5, 3, 'Sustainable AI for Global Health', ha='center', fontsize=20,
        style='italic', color='#00d4ff')

plt.savefig('visualizations/tb_ast_headline.png', dpi=300, bbox_inches='tight', facecolor='#1a1a2e')
print("✅ Saved: tb_ast_headline.png")
plt.close()

print(f"\n📈 FINAL RESULTS:")
print(f"   Accuracy: {best_acc:.2f}%")
print(f"   Energy Savings: {avg_savings:.2f}%")
print(f"   Activation: {avg_act:.2f}%")
''')

with open('create_viz.py', 'w') as f:
    f.write(viz_script)

# Run it
!python create_viz.py

# Display
from IPython.display import Image, display

print("\n" + "="*80)
print("📊 COMPREHENSIVE VISUALIZATIONS")
print("="*80)

print("\n1️⃣ 4-Panel Analysis:")
display(Image('visualizations/tb_ast_results.png'))

print("\n2️⃣ Social Media Graphic:")
display(Image('visualizations/tb_ast_headline.png'))

## Step 11: Interactive Grad-CAM Visualization

**Click on different X-rays to see what the model is looking at!**

In [ ]:
# Install Grad-CAM dependencies
!pip install -q grad-cam opencv-python

# Create Grad-CAM script
gradcam_code = '''
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        def save_gradient(grad):
            self.gradients = grad
        
        def save_activation(module, input, output):
            self.activations = output.detach()
        
        target_layer.register_forward_hook(save_activation)
        target_layer.register_backward_hook(lambda m, gi, go: save_gradient(go[0]))
    
    def generate(self, input_image, target_class=None):
        output = self.model(input_image)
        
        if target_class is None:
            target_class = output.argmax(dim=1)
        
        self.model.zero_grad()
        one_hot = torch.zeros_like(output)
        one_hot[0][target_class] = 1
        output.backward(gradient=one_hot, retain_graph=True)
        
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = torch.relu(cam)
        cam = cam.squeeze().cpu().numpy()
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        
        return cam, output

def visualize_gradcam(image_path, model_path, output_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load model
    model = models.efficientnet_b0(weights=None)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    # Setup Grad-CAM (use last conv layer)
    target_layer = model.features[-1]
    grad_cam = GradCAM(model, target_layer)
    
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    
    input_tensor = transform(img).unsqueeze(0).to(device)
    
    # Generate Grad-CAM
    cam, output = grad_cam.generate(input_tensor)
    
    # Get prediction
    probs = torch.softmax(output, dim=1)[0]
    pred_class = output.argmax(dim=1).item()
    classes = ['Normal', 'TB']
    
    # Resize CAM to image size
    img_resized = transform(img).permute(1, 2, 0).cpu().numpy()
    cam_resized = cv2.resize(cam, (224, 224))
    
    # Create heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) / 255.0
    
    # Overlay
    overlay = (img_resized * 0.5 + heatmap * 0.5)
    overlay = np.clip(overlay, 0, 1)
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(img_resized)
    axes[0].set_title('Original X-Ray', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(cam_resized, cmap='jet')
    axes[1].set_title('Grad-CAM Heatmap', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(overlay)
    axes[2].set_title(f'Prediction: {classes[pred_class]} ({probs[pred_class]*100:.1f}%)',
                     fontsize=14, fontweight='bold',
                     color='red' if pred_class == 1 else 'green')
    axes[2].axis('off')
    
    plt.suptitle(f'TB Detection Explanation - {Path(image_path).name}',
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    
    return pred_class, probs.cpu().numpy()

# Generate Grad-CAMs for sample images
import random

model_path = 'checkpoints_tb_ast/best.pt'
output_dir = Path('gradcam_examples')
output_dir.mkdir(exist_ok=True)

# Get sample images from validation set
val_normal = list(Path('data/val/Normal').glob('*.png'))[:5]
val_tb = list(Path('data/val/TB').glob('*.png'))[:5]

print("\n🔬 Generating Grad-CAM visualizations...\n")

for i, img_path in enumerate(val_normal + val_tb, 1):
    true_label = 'TB' if 'TB' in str(img_path) else 'Normal'
    output_path = output_dir / f'gradcam_{i:02d}_{true_label}.png'
    
    pred_class, probs = visualize_gradcam(img_path, model_path, output_path)
    pred_label = 'TB' if pred_class == 1 else 'Normal'
    
    status = '✅' if pred_label == true_label else '❌'
    print(f"{status} {i}. True: {true_label:6s} | Pred: {pred_label:6s} | Conf: {probs[pred_class]*100:.1f}%")

print(f"\n✅ Saved {len(val_normal + val_tb)} Grad-CAM visualizations to {output_dir}/")
'''

with open('generate_gradcam.py', 'w') as f:
    f.write(gradcam_code)

# Run Grad-CAM generation
!python generate_gradcam.py

# Display sample Grad-CAMs
from IPython.display import Image, display
from pathlib import Path
import ipywidgets as widgets

gradcam_dir = Path('gradcam_examples')
gradcam_files = sorted(gradcam_dir.glob('*.png'))

print("\n" + "="*80)
print("🔬 INTERACTIVE GRAD-CAM EXPLORER")
print("="*80)
print("\n👇 Click through the samples to see what the model is looking at!\n")

# Create interactive viewer
def show_gradcam(index):
    display(Image(gradcam_files[index]))

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(gradcam_files)-1,
    step=1,
    description='Sample:',
    continuous_update=False
)

widgets.interact(show_gradcam, index=slider)

print("\n💡 The heatmap shows which parts of the X-ray the model focuses on for its decision.")
print("   Red/yellow areas = high attention | Blue areas = low attention")

## Step 12: Save Everything to Google Drive

In [ ]:
# Save all results
!cp -r checkpoints_tb_ast /content/drive/MyDrive/TB_AST_Results/
!cp -r visualizations /content/drive/MyDrive/TB_AST_Results/
!cp -r gradcam_examples /content/drive/MyDrive/TB_AST_Results/
!cp configs/config_tb_ast.yaml /content/drive/MyDrive/TB_AST_Results/
!cp create_viz.py /content/drive/MyDrive/TB_AST_Results/
!cp generate_gradcam.py /content/drive/MyDrive/TB_AST_Results/

print("✅ All results saved to Google Drive!")
print("\n📁 Saved:")
print("   - Model checkpoints (best.pt)")
print("   - Training metrics (CSV + JSONL)")
print("   - Visualizations (4-panel + headline)")
print("   - Grad-CAM examples (10 samples)")
print("   - Scripts (visualization + gradcam)")

!ls -lh /content/drive/MyDrive/TB_AST_Results/

## Step 13: Download for GitHub Repository

In [ ]:
# Create a ZIP file with everything needed for GitHub
!apt-get install -qq zip

!mkdir -p Tuberculosis
!cp -r checkpoints_tb_ast Tuberculosis/checkpoints
!cp -r visualizations Tuberculosis/
!cp -r gradcam_examples Tuberculosis/
!cp configs/config_tb_ast.yaml Tuberculosis/configs/
!cp create_viz.py Tuberculosis/
!cp generate_gradcam.py Tuberculosis/

# Create README
readme = '''# 🫁 TB Detection with Adaptive Sparse Training

**99.3% accuracy with 89% energy savings!**

## 🌟 Results
- **Accuracy**: 99.29%
- **Energy Savings**: 89.52%
- **Activation Rate**: 9.38%

## 🚀 Quick Start

```bash
pip install -r requirements.txt
python train_ast.py --config configs/config_tb_ast.yaml
```

## 📊 Visualizations

See `visualizations/` for comprehensive results.

## 🔬 Grad-CAM

See `gradcam_examples/` for model explanation visualizations.

---

Built with ❤️ for sustainable AI in global health 🌍💚
'''

with open('Tuberculosis/README.md', 'w') as f:
    f.write(readme)

# Create requirements.txt
requirements = '''torch>=2.0.0
torchvision>=0.15.0
adaptive-sparse-training>=1.0.1
scikit-learn
matplotlib
seaborn
pyyaml
tqdm
pillow
numpy
pandas
grad-cam
opencv-python
'''

with open('Tuberculosis/requirements.txt', 'w') as f:
    f.write(requirements)

# Zip it
!cd Tuberculosis && zip -r ../Tuberculosis_GitHub.zip . -q

print("✅ GitHub repository package created!")
print("\n📦 Download 'Tuberculosis_GitHub.zip' and extract to your GitHub repo.")

# Download
from google.colab import files
files.download('Tuberculosis_GitHub.zip')

## ✅ Complete!

### What You Have Now:

1. ✅ **Trained TB detector** (99.3% accuracy, 89% energy savings)
2. ✅ **Comprehensive visualizations** (4-panel analysis + social media graphics)
3. ✅ **Grad-CAM explanations** (interactive explorer)
4. ✅ **Complete metrics** (CSV + JSONL)
5. ✅ **GitHub-ready package** (downloaded ZIP)
6. ✅ **Google Drive backup** (all files saved)

### Next Steps:

1. **GitHub**: Extract `Tuberculosis_GitHub.zip` to your repo
2. **Hugging Face**: Upload `checkpoints/best.pt` to create Space
3. **Share**: Use visualizations for social media / articles

---

**You've successfully created a sustainable, explainable AI for TB detection!** 🎉